In [1]:
#Install packages

!pip install openai
!pip install pydantic
!pip install overpy
!pip install geopy
!pip install typing
!pip install logging

In [2]:
#Import libraries
from openai import OpenAI
from pydantic import BaseModel
import random
import requests
import overpy
import math
import time
import json
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from typing import List, Dict
import numpy
import logging
import urllib.parse

C:\Users\elgin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\elgin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\elgin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
client = OpenAI(
    organization = '', #TODO: YOUR ORGANIZATION ID
    api_key = '' #TODO: YOUR API KEY
)

In [4]:
#Set Constants

email = "" #TODO: YOUR EMAIL

MAXIMUM_RADIUS_IN_KM = 5
MAXIMUM_NEARBY_PLACES = 20

# District -> Estates Postal Code mapping
estate_dict = {
    "d1": ["01", "02", "03", "04", "05", "06"],  # Raffles Place, Marina, Cecil
    "d2": ["07", "08"],                         # Chinatown, Tanjong Pagar
    "d3": ["14", "15", "16"],                   # Queenstown, Tiong Bahru, Alexandra
    "d4": ["09", "10"],                         # Sentosa, Harbourfront, Telok Blangah
    "d5": ["11", "12", "13"],                   # Pasir Panjang, Buona Vista, West Coast
    "d6": ["17"],                               # City Hall, Clarke Quay
    "d7": ["18", "19"],                         # Beach Road, Bugis, Rochor
    "d8": ["20", "21"],                         # Little India, Farrer Park
    "d9": ["22", "23"],                         # Orchard, River Valley
    "d10": ["24", "25", "26", "27"],            # Tanglin, Holland, Bukit Timah
    "d11": ["28", "29", "30"],                  # Newton, Novena
    "d12": ["31", "32", "33"],                        # Toa Payoh, Serangoon, Balestier
    "d13": ["34", "35", "36", "37"],                        # MacPherson, Potong Pasir
    "d14": ["38", "39", "40", "41"],                  # Geylang, Eunos
    "d15": ["42", "43", "44", "45"],            # Katong, Joo Chiat, Amber Road
    "d16": ["46", "47", "48"],            # Bedok, Upper East Coast
    "d17": ["49", "50", "81"],                  # Changi, Loyang
    "d18": ["51", "52"],                  # Tampines, Pasir Ris
    "d19": ["53", "54", "55", "82"],# Serangoon, Hougang, Punggol
    "d20": ["56", "57"],                        # Bishan, Ang Mo Kio
    "d21": ["58", "59"],                        # Upper Bukit Timah, Clementi Park
    "d22": ["60", "61", "62", "63", "64"],      # Jurong, Boon Lay, Tuas
    "d23": ["65", "66", "67", "68"],            # Hillview, Bukit Panjang, Choa Chu Kang
    "d24": ["69", "70", "71"],                  # Lim Chu Kang, Tengah
    "d25": ["72", "73"],                        # Woodlands, Admiralty
    "d26": ["77", "78"],                        # Upper Thomson, Springleaf
    "d27": ["75", "76"],                        # Yishun, Sembawang
    "d28": ["79", "80"]                         # Seletar, Yio Chu Kang
}

In [5]:
# Helper functions

def get_address(lat, lon):
    """
    Get address based on lat lon
    """
    nominatim_url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&zoom=18&format=jsonv2&email={email}"
    time.sleep(range(1,10))
    response = requests.get(nominatim_url)
    data = json.loads(response.text)
    post_code = data["address"]["postcode"]

    if 'error' in data:
        return None
    address = data['address']
    formatted_address = []
    for key in address:
        if "-" in key or "number" in key or "code" in key:
            continue
        formatted_address.append(address[key])

    return ', '.join(formatted_address), post_code, data["display_name"]

def geocode_address(address):
    """
    Geocode an address to latitude and longitude.
    """
    try:
        headers = {
          "email" : "elginsi2000@gmail.com"
          }
        geolocator = Nominatim(user_agent="elginsi2000@gmail.com")
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for address: {address}")
            return None, None
    except Exception as e:
        print(f"Error during geocoding: {e}")
        return None, None

def calculate_initial_compass_bearing(lat1, lon1, lat2, lon2):
    if (lat1 == lat2) and (lon1 == lon2):
        return 0
    bearing = math.atan2(math.sin(lon2-lon1)*math.cos(lat2), math.cos(lat1)*math.sin(lat2)-math.sin(lat1)*math.cos(lat2)*math.cos(lon2-lon1))
    bearing = math.degrees(bearing)
    bearing = (bearing + 360) % 360
    return bearing


def bearing_to_compass(bearing):
    directions = ["North", "North-East", "East", "South-East", "South", "South-West", "West", "North-West"]
    return directions[round(bearing/45) % 8]

def get_restaurant_details(location, price="1,2,3,4", categories="", metric="best_match"):
    #Get user location's lat long
    ONE_MAP_URL = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={location}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {
          "accept": "application/json",
          "Authorization": f"" #TODO: YOUR ONE MAP AUTHORIZATION TOKEN
      }
    response = requests.request("GET", ONE_MAP_URL, headers=headers)

    #print(response.text)
    response = response.json()
    #print(response)
    lat, lon = response["results"][0]["LATITUDE"], response["results"][0]["LONGITUDE"] #Get lat long of most relevant result
    
    # Base URL
    base_url = "https://api.yelp.com/v3/businesses/search"

    # Parameters
    params = {
        "location": location,
        "categories": categories.lower(),
        "sort_by": metric,
        "limit": 20
    }

    # Prices as separate parameters
    prices = price.split(',') if len(price) > 1 else price

    # Encode parameters
    encoded_params = urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

    # Manually add each price parameter
    price_params = ''.join(f"&price={price}" for price in prices)

    # Construct the final URL
    YELP_URL = f"{base_url}?{encoded_params}{price_params}"
    print(YELP_URL)
    headers = {
      "accept": "application/json",
      "Authorization": '' #TODO: YOUR YELP AUTHORIZATION TOKEN
    }

    response = requests.get(YELP_URL, headers=headers).json()
    #print(f"response: {response}")
    #print(f"Number of recommendations : {len(response['businesses'])}")
    formatted_responses = []
    
    try:
        for i in range(len(response["businesses"])):
            formatted_response = {}
            formatted_response["name"] = response["businesses"][i]["name"] if "name" in response["businesses"][i] else None
            formatted_response["location"] = response["businesses"][i]["location"] if "location" in response["businesses"][i] else None
            formatted_response["coordinates"] = response["businesses"][i]["coordinates"] if "coordinates" in response["businesses"][i] else None
            formatted_response["categories"] = response["businesses"][i]["categories"] if "categories" in response["businesses"][i] else None
            formatted_response["review_count"] = response["businesses"][i]["review_count"] if "review_count" in response["businesses"][i] else None
            formatted_response["price"] = response["businesses"][i]["price"] if "price" in response["businesses"][i] else None
            formatted_response["rating"] = response["businesses"][i]["rating"] if "rating" in response["businesses"][i] else None
            formatted_response["business_hours"] = response["businesses"][i]["business_hours"] if "business_hours" in response["businesses"][i] else None
            formatted_response["attributes"] = response["businesses"][i]["attributes"] if "attributes" in response["businesses"][i] else None
            formatted_response["distance"] = geodesic((lat, lon), (formatted_response["coordinates"]["latitude"], formatted_response["coordinates"]["longitude"])).km
            formatted_responses.append(formatted_response)
    except:
        print(f"Error in yelp api response: {response}")

    return formatted_responses


def parse_places_data(result, lat, lon):
    places = []
    for node in result.nodes:

        place_name = node.tags.get("name", "n/a")
        place_lat, place_lon = float(node.lat), float(node.lon)
        distance = geodesic((lat, lon), (place_lat, place_lon)).km
        bearing = calculate_initial_compass_bearing(math.radians(lat), math.radians(lon), math.radians(place_lat), math.radians(place_lon))
        compass_direction = bearing_to_compass(bearing)
        places.append([place_name, distance, compass_direction, node.tags, (place_lat,place_lon)])

    places.sort(key=lambda x: x[1])
    for place in places[:MAXIMUM_NEARBY_PLACES]:
      restaurant_details = get_restaurant_details(place[4][0], place[4][1])
      place.append(restaurant_details)


    return places


def get_nearby_places_amenity(address, amenity):
    #node(around:{MAXIMUM_RADIUS_IN_KM * 1000},{lat},{lon})["name"]["place"];
    
    lat, lon = geocode_address(address)
    if lat is None or lon is None:
        return "Unable to geocode the provided address."
    
    query = f"""
    (
        node[amenity="{amenity}"](around:{MAXIMUM_RADIUS_IN_KM * 1000},{lat},{lon})["name"];
        >;
    );
    out meta;
    """
    result = overpy.Overpass().query(query)
    #print(f"result: {result}")

    places = parse_places_data_amenity(result, lat, lon)

    return places[:MAXIMUM_NEARBY_PLACES]

def parse_places_data_amenity(result, lat, lon):
    places = []
    for node in result.nodes:

        place_name = node.tags.get("name", "n/a")
        place_lat, place_lon = float(node.lat), float(node.lon)
        distance = geodesic((lat, lon), (place_lat, place_lon)).km
        bearing = calculate_initial_compass_bearing(math.radians(lat), math.radians(lon), math.radians(place_lat), math.radians(place_lon))
        compass_direction = bearing_to_compass(bearing)
        places.append(
            {
                "name": place_name, 
                "distance": distance,
                "coordinates": {"latitude": place_lat, "longitutude": place_lon},
                "attributes": node.tags
            }
        )

    places.sort(key=lambda x: x["distance"])


    return places


In [6]:
geocode_address("The Fullerton Hotel, 049178")

(1.28620185, 103.85307262998235)

In [14]:
get_nearby_places_amenity("The Fullerton Hotel, 049178", 'bank')

{'alt_name': 'POSB', 'amenity': 'bank', 'atm': 'yes', 'name': 'DBS'}
{'amenity': 'bank', 'brand': 'BOC', 'brand:wikidata': 'Q790068', 'brand:wikipedia': 'en:Bank of China', 'name': 'BOC', 'official_name': 'Bank of China'}
{'amenity': 'bank', 'name': 'Indian Overseas Bank'}
{'amenity': 'bank', 'name': 'Rashid Hussein Bank'}
{'amenity': 'bank', 'atm': 'yes', 'drive_through': 'no', 'level': '-1', 'name': 'DBS', 'opening_hours': 'Mo-Fr 10:00-18:00, Sa 10:00-14:30', 'phone': '1800 111 1111', 'website': 'https://www.dbs.com.sg', 'wheelchair': 'yes', 'wikidata': 'Q342642', 'wikipedia': 'en:DBS'}
{'addr:city': 'Singapore', 'addr:housenumber': '10', 'addr:postcode': '307506', 'addr:street': 'Sinaran Drive', 'addr:unit': '#02-19/20', 'alt_name': 'POSB', 'amenity': 'bank', 'atm': 'yes', 'branch': 'Square 2', 'brand': 'POSB', 'brand:wikidata': 'Q7120030', 'brand:wikipedia': 'en:POSB Bank', 'check_date:opening_hours': '2023-09-08', 'level': '1', 'name': 'POSB', 'opening_hours': 'Mo-Fr 08:30-16:00; 

[{'name': 'IG',
  'distance': 0.1398219252314539,
  'coordinates': {'latitude': 1.2850662, 'longitutude': 103.8525201},
  'attributes': {'amenity': 'bank',
   'name': 'IG',
   'source': 'Kaart Ground Survey 2017'}},
 {'name': 'HL Bank',
  'distance': 0.16426280203698554,
  'coordinates': {'latitude': 1.2847339, 'longitutude': 103.8528462},
  'attributes': {'addr:housenumber': '20',
   'addr:street': 'Collyer Quay',
   'amenity': 'bank',
   'name': 'HL Bank',
   'source': 'Kaart Ground Survey 2017'}},
 {'name': 'Standard Chartered',
  'distance': 0.1850779778263378,
  'coordinates': {'latitude': 1.2852425, 'longitutude': 103.8517099},
  'attributes': {'addr:street': 'Battery Road',
   'amenity': 'bank',
   'brand': 'Standard Chartered',
   'brand:wikidata': 'Q548278',
   'brand:wikipedia': 'en:Standard Chartered',
   'name': 'Standard Chartered',
   'source': 'Kaart Ground Survey 2017'}},
 {'name': 'BNP Paribas',
  'distance': 0.1947079451554284,
  'coordinates': {'latitude': 1.2844674,

In [72]:
#What are the top-rated Japanese restaurant at Hilton Garden Inn Singapore Serangoon, 219924 located within district 8
get_restaurant_details("Hilton Garden Inn Singapore Serangoon, 219924", "1,2,3,4", "Japanese", "rating")

https://api.yelp.com/v3/businesses/search?location=Hilton%20Garden%20Inn%20Singapore%20Serangoon%2C%20219924&categories=japanese&sort_by=rating&limit=20&price=1&price=2&price=3&price=4


[{'name': 'Sushi Ichi',
  'location': {'address1': '320 Orchard Rd',
   'address2': '01-04',
   'address3': '',
   'city': 'Singapore',
   'zip_code': '238865',
   'country': 'SG',
   'state': 'SG',
   'display_address': ['320 Orchard Rd',
    '01-04',
    'Singapore 238865',
    'Singapore']},
  'coordinates': {'latitude': 1.3051, 'longitude': 103.83275},
  'categories': [{'alias': 'japanese', 'title': 'Japanese'},
   {'alias': 'sushi', 'title': 'Sushi Bars'}],
  'review_count': 5,
  'price': '$$$$',
  'rating': 5.0,
  'business_hours': [],
  'attributes': {'business_temp_closed': None,
   'menu_url': None,
   'open24_hours': None,
   'waitlist_reservation': None},
  'distance': 2.169822593407615},
 {'name': 'Sanji Kagoshima Ramen',
  'location': {'address1': 'Orchid Hotel',
   'address2': '#01-14, 1 Tras Link',
   'address3': '',
   'city': 'Singapore',
   'zip_code': '078867',
   'country': 'SG',
   'state': 'SG',
   'display_address': ['Orchid Hotel',
    '#01-14, 1 Tras Link',
   

In [7]:
poi = ['bank', 'hospital']#'restaurant', 
district = list(range(1,29))
# address_dict = {
#     1: "The Fullerton Hotel, 049178",  # District 01: Raffles Place, Cecil, Marina
#     2: "International Plaza, 079903",       # District 02: Anson, Tanjong Pagar
#     3: "Great World City, 237994",   # District 03: Queenstown, Tiong Bahru
#     4: "VivoCity, 098585",            # District 04: Telok Blangah, Harbourfront
#     5: "Fusionopolis, 138632",         # District 05: Pasir Panjang, Clementi New Town
#     6: "High Street Centre, 179094",  # District 06: High Street, Beach Road
#     7: "Golden Mile Tower, 199589",       # District 07: Middle Road, Golden Mile
#     8: "Hilton Garden Inn Singapore Serangoon, 219924",  # District 08: Little India
#     9: "ION Orchard, 238801",              # District 09: Orchard, Cairnhill, River Valley
#     10: "British High Commission, 247919",  # District 10: Ardmore, Bukit Timah, Holland Road
#     11: "Novena Square, 307684",         # District 11: Watten Estate, Novena, Thomson
#     12: "HDB Hub, 310530",         # District 12: Balestier, Toa Payoh, Serangoon
#     13: "Kallang MRT Station, 387405", # District 13: Macpherson, Braddell
#     14: "Paya Lebar Square, 409051",   # District 14: Geylang, Eunos
#     15: "i12 Katong, 428802",         # District 15: Katong, Joo Chiat, Amber Road
#     16: "Bedok Mall, 467360",   # District 16: Bedok, Upper East Coast
#     17: "UE BizHub EAST, 486017",  # District 17: Loyang, Changi
#     18: "Tampines Mall, 529510",     # District 18: Tampines, Pasir Ris
#     19: "Hougang Mall, 538766",      # District 19: Serangoon Garden, Hougang, Punggol
#     20: "AMK Hub, 569933",         # District 20: Bishan, Ang Mo Kio
#     21: "Bukit Timah Plaza, 588996",   # District 21: Upper Bukit Timah, Clementi Park
#     22: "JEM, 608549",             # District 22: Jurong
#     23: "Lot One, 689812",      # District 23: Hillview, Dairy Farm, Bukit Panjang
#     24: "Fajar Shopping Centre, 670445",     # District 24: Lim Chu Kang, Tengah
#     25: "Causeway Point, 738099",      # District 25: Kranji, Woodgrove
#     26: "Thomson Plaza, 574408",   # District 26: Upper Thomson, Springleaf
#     27: "Northpoint City, 769098",     # District 27: Yishun, Sembawang
#     28: "Seletar Mall, 797653"    # District 28: Seletar
# }

address_dict = {
    1: "Marina Bay Sands, 018956",  # District 01: Raffles Place, Cecil, Marina
    2: "Guoco Tower, 078881",  # District 02: Anson, Tanjong Pagar
    3: "Alexandra Retail Centre, 119963",  # District 03: Queenstown, Tiong Bahru
    4: "HarbourFront Centre, 099253",  # District 04: Telok Blangah, Harbourfront
    5: "West Coast Plaza, 127371",  # District 05: Pasir Panjang, Clementi New Town
    6: "Raffles City Tower, 179101",  # District 06: High Street, Beach Road
    7: "Bugis Junction, 188021",  # District 07: Middle Road, Golden Mile
    8: "Mustafa Centre, 207704",  # District 08: Little India
    9: "Ngee Ann City, 238873",  # District 09: Orchard, Cairnhill, River Valley
    10: "One Holland Village, 275748",  # District 10: Ardmore, Bukit Timah, Holland Road
    11: "United Square Shopping Mall, 307591",  # District 11: Watten Estate, Novena, Thomson
    12: "Zhongshan Mall, 329984",  # District 12: Balestier, Toa Payoh, Serangoon
    13: "MacPherson Mall, 368125",  # District 13: Macpherson, Braddell
    14: "Kinex Mall, 437157",  # District 14: Geylang, Eunos
    15: "Parkway Parade, 449269",  # District 15: Katong, Joo Chiat, Amber Road
    16: "Eastpoint Mall, 528833",  # District 16: Bedok, Upper East Coast
    17: "Changi City Point, 486038",  # District 17: Loyang, Changi
    18: "White Sands, 518457",  # District 18: Tampines, Pasir Ris
    19: "NEX, 556083",  # District 19: Serangoon Garden, Hougang, Punggol
    20: "Junction 8, 579837",  # District 20: Bishan, Ang Mo Kio
    21: "Bukit Timah Shopping Centre, 588179",  # District 21: Upper Bukit Timah, Clementi Park
    22: "Westgate, 608532",  # District 22: Jurong
    23: "Hillion Mall, 678278",  # District 23: Hillview, Dairy Farm, Bukit Panjang
    24: "Sungei Kadut Industrial Estate, 729340",  # District 24: Lim Chu Kang, Tengah
    25: "Woodlands Civic Centre, 730900",  # District 25: Kranji, Woodgrove
    26: "Sembawang Shopping Centre, 758459",  # District 26: Upper Thomson, Springleaf
    27: "YewTee Point, 689578",  # District 27: Yishun, Sembawang
    28: "Greenwich V, 807011"  # District 28: Seletar
}

eval_address_dict = {
    1: "One Raffles Place, 048616",  # District 01: Raffles Place, Cecil, Marina
    2: "AXA Tower, 068811",  # District 02: Anson, Tanjong Pagar
    3: "Queensway Shopping Centre, 149053",  # District 03: Queenstown, Tiong Bahru
    4: "Mount Faber Peak, 099203",  # District 04: Telok Blangah, Harbourfront
    5: "National University of Singapore, 119077",  # District 05: Pasir Panjang, Clementi New Town
    6: "The Adelphi, 179803",  # District 06: High Street, Beach Road
    7: "DUO Tower, 189352",  # District 07: Middle Road, Golden Mile
    8: "Tekka Centre, 210665",  # District 08: Little India
    9: "Paragon Shopping Centre, 238859",  # District 09: Orchard, Cairnhill, River Valley
    10: "Dempsey Hill, 249679",  # District 10: Ardmore, Bukit Timah, Holland Road
    11: "Velocity @ Novena Square, 307683",  # District 11: Watten Estate, Novena, Thomson
    12: "Balestier Point, 329727",  # District 12: Balestier, Toa Payoh, Serangoon
    13: "Potong Pasir Community Club, 358361",  # District 13: Macpherson, Braddell
    14: "Tanjong Katong Complex, 400845",  # District 14: Geylang, Eunos 
    15: "Joo Chiat Complex, 420001",  # District 15: Katong, Joo Chiat, Amber Road 
    16: "Bedok Reservoir Park, 479245",  # District 16: Bedok, Upper East Coast 
    17: "Changi Airport Terminal 3, 819663",  # District 17: Loyang, Changi
    18: "Our Tampines Hub, 528523",  # District 18: Tampines, Pasir Ris
    19: "Waterway Point, 828761",  # District 19: Serangoon Garden, Hougang, Punggol
    20: "Bishan Community Club, 579799",  # District 20: Bishan, Ang Mo Kio
    21: "Beauty World Centre, 588177",  # District 21: Upper Bukit Timah, Clementi Park
    22: "IMM, 609601",  # District 22: Jurong
    23: "Rail Mall, 678040",  # District 23: Hillview, Dairy Farm, Bukit Panjang
    24: "Kranji War Memorial, 738656",  # District 24: Lim Chu Kang, Tengah 
    25: "Woodlands Regional Library, 730900",  # District 25: Kranji, Woodgrove
    26: "Upper Thomson Plaza, 574408",  # District 26: Upper Thomson, Springleaf
    27: "Sun Plaza, 757713",  # District 27: Yishun, Sembawang
    28: "Seletar Aerospace Park, 798386"  # District 28: Seletar
}

In [8]:
#20 types

question_bank = {
    "restaurant" : [
        "Where is the nearest {poi} within district {district} at {address}?",
        "What are high ratings {poi} within district {district} at {address}?",
        "Which {poi} has the best reviews in district {district} near {address}?",
        "What are the opening hours of the nearest {poi} in district {district} at {address}?",
        "Where are the nearest {poi} that is open at 1pm in district {district} at {address}?",
        "Which {poi} has at least 30 review counts within district {district} at {address}?",
        "What is the most popular {poi} in district {district} at {address}?",
        "Where can I find a cheap {poi} at {address} in district {district}?",
        "Are there any new {poi} openings in district {district} at {address}?",
        "Which {poi} has the shortest waiting time in district {district} at {address}?",
        "Where can I find a budget-friendly {poi} in district {district} at {address}?",
        "Are there any nearby {poi} with rating above 4.5 within district {district} at {address}",
        "Which {poi} in district {district} has the best customer service at {address}?",
        "Are there any pet-friendly {poi} within district {district} at {address}?",
        "Where is the closest wheelchair-accessible {poi} in district {district} at {address}?",
        "Suggest a thai {poi} at {address} located within district {district} with a high number of reviews.",
        "Find a dessert {poi} at {address} located within district {district} with over 50 reviews.",
        "Any chinese {poi} that have at least a 3.9 rating, located within district {district} at {address}?",
        "What are the top-rated japanese {poi} at {address} located within district {district}",
        "Find a seafood {poi} in district {district} at {address} that is open until midnight",
        "Suggest any bar {poi} in district {district} at {address} that is open until at least 1 AM?"
    ],
    
    "bank": [
        "Where is the nearest {poi} within district {district} at {address}?",
        "What are the opening hours of the nearest {poi} at {address} in district {district}?",
        "Where is the nearest OCBC {poi} branch within district {district} at {address}?",
        "Where are available DBS {poi} branch within district {district} at {address}?",
        "Where can I deposit cash for my Standard Chartered {poi} account in district {district} at {address}?"
    ],
    
     "hospital": [
        "Where is the nearest {poi} within district {district} at {address}?",
        "What are the opening hours of the nearest {poi} at {address} in district {district}?",
        "Where is the nearest {poi} with a cardiology department within district {district} at {address}?",
        "Where are {poi} in district {district} at {address} with wheelchair accessibility?"
    ]
    
}

In [9]:
def get_question(question_bank, poi, district, address_dict):
    poi_selected = random.choice(poi)
    #poi_selected= numpy.random.choice(poi, 1, [0.8, 0.2, 0.1])[0]
    district_selected = random.choice(district)
    address_selected = address_dict[district_selected]
    qn = random.choice(question_bank[poi_selected])
    final_qn = qn.format(poi = poi_selected, district = district_selected, address = address_selected)
    return final_qn

#Test function
get_question(question_bank, poi, district, address_dict)

'Where are available DBS bank branch within district 26 at Sembawang Shopping Centre, 758459?'

In [10]:
#Answer structure for restaurant
class RestaurantAnswerFormat(BaseModel):
    rank: int
    distance_in_km: float
    name: str
    location: str
    coordinates: List[str]
    price: str
    categories: str
    opening_hours: str
    rating: str
    review_count: int
    district: int
    attributes: str

class RestaurantAnswerFormatList(BaseModel):
    result: List[RestaurantAnswerFormat]
        
#Answer structure for banks
class AmenityAnswerFormat(BaseModel):
    rank: int
    distance_in_km: float
    name: str
    location: str
    coordinates: List[str]
    opening_hours: str
    district: int
    attributes: str

class AmenityAnswerFormatList(BaseModel):
    result: List[AmenityAnswerFormat]

In [11]:
api_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_restaurant_details",
            "description": "Retrieve details of the nearest restaurants based on location using Yelp API.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The address of the location to search for restaurants, excluding the district. Example: '522 Woodlands Drive 14, Block 522, Singapore 730522'."
                    },
                     "amenity": {
                        "type": "string",
                        "description": "The type of amenity. Example: 'restaurant', 'bank', 'hospital'"
                    },
                    "price": {
                        "type": "string",
                        "description": "The price range for restaurants. Default should be '1,2,3,4', unless otherwise hinted (ie cheapest etc). Example: '1' (cheapest),'2,3' (moderate), '4'(expensive)."
                    },
                    "categories": {
                        "type": "string",
                        "description": "The type of restaurants. Default should be "", unless otherwise hinted. Example: 'Japanese'."
                    },
                    "metric": {
                        "type": "string",
                        "description": " Attribute to sort results by. Default should be 'best_match' unless otherwise hinted. Metric must be either 'distance' or 'rating' or 'review_count' or 'best_match'. If metric is not a valid word, default to 'best-match'."
                    }
                },
                "required": ["location", "amenity", "price", "categories", "metric"],
                "additionalProperties": False,
                #"strict": True
            },
            "strict": True
            
        }
    },
    
    {
        "type": "function",
        "function": {
            "name": "get_nearby_places_amenity",
            "description": "Retrieve details of the nearest amenity based on location using Overpass API.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The address of the location to search for amenity, excluding the district. Example: '522 Woodlands Drive 14, Block 522, Singapore 730522'."
                    },
                    "amenity": {
                        "type": "string",
                        "description": "The type of amenity. Example: 'restaurant', 'bank', 'hospital'"
                    }
                },
                "required": ["location", "amenity"],
                "additionalProperties": False,
                #"strict": True
            },
            "strict": True
            
        }
    }
]


In [12]:
messages = [
    {"role": "system", "content": 
     """
        You are a helpful assistant and you are assigned the task is to format the results returned by the tool. 
        The values should be an exact match with the results returned from the tools.
        The results are already ordered according to the metric prompted by the user. 
        Return at least 15 results for restaurants and at least 5 results for banks or hospitals.

    """
    }
]

NUM_SAMPLES = 120
MAX_RETRY = 30
i = 0
retry = 0
while i < NUM_SAMPLES and retry < MAX_RETRY:
    try:
        #Clear message array
        messages.clear()
        
        #Get questions
        sample_qn = get_question(question_bank, poi, district, eval_address_dict)
        messages.append({"role": "user", "content": sample_qn})

        # Ensure tool is triggered
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=api_tools,
            #tool_choice= "required"  # Ensures the model can call tools when necessary
            tool_choice = {"type": "function", "function": {"name": "get_nearby_places_amenity"}}
        )


        # Check if tool_calls exist before accessing
        tool_calls = completion.choices[0].message.tool_calls
        #print(tool_calls)
        
        if tool_calls is not None and len(tool_calls) > 0:
            tool_call = tool_calls[0]
            args = json.loads(tool_call.function.arguments)
            print(f"Arguments: {args}")
        
            #result = get_restaurant_details(args["location"], args["price"], args["categories"], args["metric"])
            #result = get_nearby_places_amenity(args["location"], args["amenity"])
            
            if (args["amenity"] == "restaurant"):
                result = get_restaurant_details(args["location"], args["price"], args["categories"], args["metric"])
            else:
                result = get_nearby_places_amenity(args["location"], args["amenity"])
                
            if len(result) == 0:
                raise Exception("Overpass api failed to retrive data")
            #print(f'Results: {json.dumps(result, indent=4)}')

            # Append the model's function call message
            messages.append(completion.choices[0].message)

            #for i in range(len(result)):
            # Append result message
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result)
            })
            
            #print(messages)

            # Generate the final response
#             completion_2 = client.beta.chat.completions.parse(
#                     model="gpt-4o-mini",
#                     messages=messages,
#                     #tools=api_tools,
#                     temperature= 0.3, #control randomess of output
#                     #max_tokens=256,
#                     top_p = 0.8, #ignores the least likely 20% subset of words
#                     response_format= AmenityAnswerFormatList
#             )
            if (args["amenity"] == "restaurant"):
                completion_2 = client.beta.chat.completions.parse(
                    model="gpt-4o-mini",
                    messages=messages,
                    #tools=api_tools,
                    temperature= 0.3, #control randomess of output
                    #max_tokens=256,
                    top_p = 0.8, #ignores the least likely 20% subset of words
                    response_format= RestaurantAnswerFormatList
                )
            else: #banks, hospitals
                 completion_2 = client.beta.chat.completions.parse(
                    model="gpt-4o-mini",
                    messages=messages,
                    #tools=tools,
                    temperature= 0.3, #control randomess of output
                    #max_tokens=256,
                    top_p = 0.8, #ignores the least likely 20% subset of words
                    response_format= AmenityAnswerFormatList
                )

            event = completion_2.choices[0].message.parsed

            # Convert dictionary to JSON string for better readability
            formatted_result = f"{sample_qn}, {json.dumps([json.dumps(res, default=lambda o: o.__dict__) for res in event])}\n"

            # Write to a .txt file
            with open("eval_output.txt", "a", encoding="utf-8") as file:
                file.write(formatted_result)

            # Log progress
            i += 1
            progress = math.floor((i*100)/NUM_SAMPLES)
            print(f"Progress completed: {progress}%")
            
        else:
            print("❌ No tool calls were made by the model. Double-check the prompt and tool definitions.")
    
    except:
        
        logging.exception('')
        
        with open("failed_qn.txt", "a", encoding="utf-8") as file:
            file.write(f"{sample_qn}\n")
            
        retry += 1
        continue
    
print("File successfully written.")    

Arguments: {'location': 'Seletar Aerospace Park, 798386', 'amenity': 'bank'}
Progress completed: 0%
Arguments: {'location': 'Queensway Shopping Centre, 149053', 'amenity': 'bank'}
Progress completed: 1%
Arguments: {'location': 'IMM, 609601', 'amenity': 'bank'}
Progress completed: 2%
Arguments: {'location': 'AXA Tower, 068811', 'amenity': 'hospital'}
Progress completed: 3%
Arguments: {'location': 'Queensway Shopping Centre, 149053', 'amenity': 'bank'}
Progress completed: 4%
Arguments: {'location': 'Joo Chiat Complex, 420001', 'amenity': 'hospital'}
Progress completed: 5%
Arguments: {'location': 'Woodlands Regional Library, 730900', 'amenity': 'bank'}
Progress completed: 5%
Arguments: {'location': 'Dempsey Hill, 249679', 'amenity': 'hospital'}
Progress completed: 6%
Arguments: {'location': 'IMM, 609601', 'amenity': 'bank'}
Progress completed: 7%
Arguments: {'location': 'Kranji War Memorial, 738656', 'amenity': 'hospital'}
Progress completed: 8%
Arguments: {'location': 'Queensway Shopping

ERROR:root:
Traceback (most recent call last):
  File "<ipython-input-12-dd1e6886d178>", line 56, in <module>
    raise Exception("Overpass api failed to retrive data")
Exception: Overpass api failed to retrive data


Arguments: {'location': 'Tekka Centre, 210665', 'amenity': 'hospital'}
Progress completed: 16%
Arguments: {'location': 'Joo Chiat Complex, 420001', 'amenity': 'hospital'}
Progress completed: 17%
Arguments: {'location': 'Balestier Point, 329727', 'amenity': 'hospital'}
Progress completed: 18%
Arguments: {'location': 'Kranji War Memorial', 'amenity': 'bank'}
Progress completed: 19%
Arguments: {'location': 'Bedok Reservoir Park, 479245', 'amenity': 'hospital'}
Progress completed: 20%
Arguments: {'location': 'Waterway Point, 828761', 'amenity': 'hospital'}
Progress completed: 20%
Arguments: {'location': 'Dempsey Hill, 249679', 'amenity': 'bank'}
Progress completed: 21%
Arguments: {'location': 'Velocity @ Novena Square, 307683', 'amenity': 'bank'}
Progress completed: 22%
Arguments: {'location': 'Dempsey Hill, 249679', 'amenity': 'bank'}
Progress completed: 23%
Arguments: {'location': 'Mount Faber Peak, 099203', 'amenity': 'bank'}
Progress completed: 24%
Arguments: {'location': 'Upper Thomso

ERROR:root:
Traceback (most recent call last):
  File "<ipython-input-12-dd1e6886d178>", line 48, in <module>
    result = get_nearby_places_amenity(args["location"], args["amenity"])
  File "<ipython-input-5-cbb4007ca53c>", line 212, in get_nearby_places_amenity
    result = overpy.Overpass().query(query)
  File "C:\Users\elgin\anaconda3\lib\site-packages\overpy\__init__.py", line 189, in query
    raise current_exception
overpy.exception.OverpassGatewayTimeout: Server load too high


Arguments: {'location': 'The Adelphi, 179803', 'amenity': 'bank'}
Progress completed: 35%
Arguments: {'location': 'National University of Singapore, 119077', 'amenity': 'bank'}


...

Geocoding failed for address: Paragon Shopping Centre, 238859
Progress completed: 82%
Arguments: {'location': 'Paragon Shopping Centre, 238859', 'amenity': 'hospital'}
Geocoding failed for address: Paragon Shopping Centre, 238859
Progress completed: 83%
Arguments: {'location': 'DUO Tower, 189352', 'amenity': 'bank'}
Progress completed: 84%
Arguments: {'location': 'Tekka Centre, 210665', 'amenity': 'hospital'}
Progress completed: 85%
Arguments: {'location': 'Tanjong Katong Complex, 400845', 'amenity': 'bank'}
Progress completed: 85%
Arguments: {'location': 'Velocity @ Novena Square, 307683', 'amenity': 'bank'}
Progress completed: 86%
Arguments: {'location': 'Sun Plaza, 757713', 'amenity': 'bank'}
Progress completed: 87%
Arguments: {'location': 'Tekka Centre, 210665', 'amenity': 'bank'}
Progress completed: 88%
Arguments: {'location': 'Bedok Reservoir Park, 479245', 'amenity': 'bank'}
Progress completed: 89%
Arguments: {'location': 'Upper Thomson Plaza, 574408', 'amenity': 'hospital'}
Geo

ERROR:root:
Traceback (most recent call last):
  File "<ipython-input-12-dd1e6886d178>", line 56, in <module>
    raise Exception("Overpass api failed to retrive data")
Exception: Overpass api failed to retrive data


Arguments: {'location': 'Velocity @ Novena Square, 307683', 'amenity': 'hospital'}
Progress completed: 95%
Arguments: {'location': 'Beauty World Centre, 588177', 'amenity': 'bank'}
Progress completed: 95%
Arguments: {'location': 'Kranji War Memorial, 738656', 'amenity': 'hospital'}
Progress completed: 96%
Arguments: {'location': 'Tekka Centre, 210665', 'amenity': 'bank'}
Progress completed: 97%
Arguments: {'location': 'IMM, 609601', 'amenity': 'hospital'}


ERROR:root:
Traceback (most recent call last):
  File "<ipython-input-12-dd1e6886d178>", line 56, in <module>
    raise Exception("Overpass api failed to retrive data")
Exception: Overpass api failed to retrive data


Arguments: {'location': 'Kranji War Memorial, 738656', 'amenity': 'hospital'}
Progress completed: 98%
Arguments: {'location': 'Potong Pasir Community Club, 358361', 'amenity': 'hospital'}
Progress completed: 99%
Arguments: {'location': 'Paragon Shopping Centre, 238859', 'amenity': 'bank'}
Geocoding failed for address: Paragon Shopping Centre, 238859
Progress completed: 100%
File successfully written.
